# Jupyter Snake

---

In this project our goal is to beat the game of snake, applying multiple RL techniques in order to teach an agent how to play the game.\
The project is divided in 3 parts:
1. Developement of the Environment
2. Implementation of the Algorithms
3. Learning and evaluating phase of the Algorithms

This whole project is developed as final project for the "Reinforcement Learning" course (2024-2025).

Authors : *Bredariol Francesco, Savorgnan Enrico, Tic Ruben*

In [2]:
from algorithms import *
from eligibility_traces import *
from epsilon_scheduler import * 
from snake_environment import *
from states_bracket import *
from utils import *

## PART 1
---
*Environment*

##### **The Game**

For who who doesn't know Snake is a not just a game but a genre of action video games.\
It was born in 1976 competitive arcade video game Blockade, where the goal was to survive longer than others players while collecting the most possible food.\
In this game you control the head of a snake on grid world and you aim to eat some food in order to become bigger. The big difficulty here is that if you hit your tail (this is the only common rule for all snake variant) you die.\
There are multiple version of the game and some of them are really weird (where teleportation can occour or some food can actually make you die).\
We took in account the most basic version, where:

1. The world is a discrete grid world of size $n\times m$
2. There is always only one food (apple) on the grid world, until you eat it and it changes position
3. There are no periodic boundary conditions, meaning that if you hit a wall you die

The rest is just as described as in the introduction to the game.\
Little side note is that this version is inspired by the "Snake Byte" published by Sirius Software in 1982.

##### **The Implementation**

Thanks to Gymnasium and PyGame the implementation of this simple version of the game is pretty straightforward.\
Developed completely in Python in the file "snake_environment.py", this implementation follows the Gym protocol, defining:

1. Step
2. Reset
3. Render, which use PyGame
4. Close

All others functions are private and only used inside the class, for the exception of "get_possible_action" which is useful for our purpose.

One important thing is that we actually defined a maximum number of step inside the environment to prevent infinte loop driven by bad policies while training. This is a parameter for the __init__ with default value 1000.

In [13]:
import random
env = SnakeEnv(render_mode="human")

done, keep = False, True

state, _ = env.reset()
action = 0

while not done and keep:
    action = random.choice(env.get_possible_actions(action))
    state, reward, done, trunc, inf = env.step(action)
    keep = env.render()

env.close()

##### **The Dimensionality Problem**

Once the environment is defined, one can think about how big the space of all possible configuration is.\
Well, doing some (pretty bad but although valid) approximation, the dimension of all possible configuration ends up being something like this:
$$
    |S| = (n\times m)(n\times m)2^{(n\times m)}
$$
This should describe all the possible positions of the apple, all the possible position of the head and all possible configuration on the grid of the tails (now this is the big approximation, since the tail configuration is not indepent from the head position). Anyway, even if this is an approximation one can simply add the "blocks" on the grid world (static cells that kill, if touched, the snake) and the dimension should exactly being that big.

Now this is not a simple thing to deal with while learning. Solution? Bidding (or bracketing, how we actually call it).

## PART 2
---
*Algorithms*